In [1]:
!apt-get install -y tesseract-ocr
!pip install -q pytesseract pillow sentence-transformers docx2txt pandas scikit-learn tqdm gradio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [2]:
import pytesseract
from PIL import Image
import docx2txt
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
import os


In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
def extract_resume_text(file_path):
    try:
        if file_path.endswith(".docx"):
            return docx2txt.process(file_path)

        elif file_path.endswith(".txt"):
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                return f.read()

        elif file_path.lower().endswith((".png", ".jpg", ".jpeg")):
            image = Image.open(file_path)
            return pytesseract.image_to_string(image)

        else:
            return ""
    except:
        return ""


In [5]:
MATCH_THRESHOLD = 80
MIN_TEXT_LENGTH = 30

def process_resumes(files, job_description):
    results = []

    job_embedding = model.encode(job_description)

    for file in files:
        text = extract_resume_text(file.name).lower().strip()

        if len(text) < MIN_TEXT_LENGTH:
            results.append({
                "Resume Name": os.path.basename(file.name),
                "Match Percentage": 0.0
            })
            continue

        resume_embedding = model.encode(text)
        score = cosine_similarity(
            [resume_embedding], [job_embedding]
        )[0][0]

        results.append({
            "Resume Name": os.path.basename(file.name),
            "Match Percentage": round(score * 100, 2)
        })

    df = pd.DataFrame(results).sort_values(
        by="Match Percentage", ascending=False
    )

    summary = (
        f"Total resumes processed : {len(df)}\n"
        f"Resumes Selected (≥80%) : {len(df[df['Match Percentage'] >= MATCH_THRESHOLD])}\n"
        f"Resumes Rejected (<80%) : {len(df[df['Match Percentage'] < MATCH_THRESHOLD])}"
    )


In [ ]:
interface = gr.Interface(
    fn=process_resumes,
    inputs=[
        gr.File(
            file_types=[".png", ".jpg", ".jpeg", ".docx", ".txt"],
            file_count="multiple",
            label="Upload Resumes"
        ),
        gr.Textbox(
            lines=4,
            placeholder="Enter Job Description here...",
            label="Job Description"
        )
    ],
    outputs=[
        gr.Dataframe(
            label="Resume Match Results",
            interactive=False
        ),
        gr.Textbox(
            label="Summary"
        )
    ],
    title="AI Resume Screening System",
    description=(
        "OCR + Semantic AI based resume matcher. "
        "Supports scanned resumes, DOCX, and TXT files."
    )
)

interface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6b4b32e3ac0e6a827d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
